In [2]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import matplotlib as mpl
import mne
import pathlib

In [3]:
lemon_info = pd.read_csv(
  "/data0/practical-sose23/brain-age/data/lemon/Behavioural_Data_MPILMBB_LEMON/META_File_IDs_Age_Gender_Education_Drug_Smoke_SKID_LEMON.csv")

### Interface with data

In [4]:
data_dir = pathlib.Path("/data0/practical-sose23/brain-age/data/bap/preprocessed/")
assert data_dir.is_dir()

data_dir_healthy = data_dir / "healthy_controls/preprocessed"
assert data_dir_healthy.is_dir()

data_dir_chronic_pain = data_dir / "chronic_pain_patients"
assert data_dir_chronic_pain.is_dir()

In [ ]:
eeg_files_all = list(data_dir.rglob("*.vhdr"))

eeg_files_raw = [f for f in eeg_files_all if not "preprocessed" in str(f)]
eeg_files_preprocessed = [f for f in eeg_files_all if "preprocessed" in str(f)]

eeg_files_healthy = list(data_dir_healthy.rglob("*.vhdr"))
eeg_files_chronic_pain = [f for f in eeg_files_preprocessed if "chronic_pain_patients" in str(f)]


### Analyze the subject data

#### Load clinical data

In [ ]:
f_subj = data_dir / "clinical_data_updated_2020-08-04.ods"
assert f_subj.is_file()

In [ ]:
df_subj_chronic_pain = pd.read_excel(f_subj, sheet_name=0, engine="odf", skiprows=1)
df_subj_healthy = pd.read_excel(f_subj, sheet_name=1, engine="odf", skiprows=0)

df_subj = pd.concat([df_subj_chronic_pain, df_subj_healthy])
df_subj["Age(years)"] = df_subj["Age(years)"].fillna(df_subj["Age (years)"])

print(f"# recorded subjects:      {len(df_subj_healthy)+len(df_subj_chronic_pain)}")
print(f"# raw eeg files:          {len(eeg_files_raw)}")
print(f"# preprocessed eeg files: {len(eeg_files_preprocessed)}")

zfill_ints = lambda x:str(x).zfill(3) if type(x) else x
df_subj_chronic_pain["Subject ID"] = df_subj_chronic_pain["Subject ID"].astype(str).map(zfill_ints)
df_subj_healthy["Subject ID"] = df_subj_healthy["Subject ID"].astype(str).map(zfill_ints)
df_subj["Subject ID"] = df_subj["Subject ID"].astype(str).map(zfill_ints)

df_subj

In [ ]:
len(df_subj)

In [ ]:
df_subj.hist(figsize=(15,15))

#### Sex imbalance: more women among patients

In [ ]:
plt.figure(figsize=(8,4))

plt.subplot(1,2,1)
plt.title("Healthy Controls")#, fontsize=16)
df_subj_healthy[df_subj_healthy["Sex (m/f)"]=="m"]["Age (years)"].hist(
    histtype="step", label="male", color="b", linewidth=3, density=False)
df_subj_healthy[df_subj_healthy["Sex (m/f)"]=="f"]["Age (years)"].hist(
    histtype="step", label="female", color="r", linewidth=3, density=False)
plt.ylabel("# participants")#, fontsize=14)
plt.xlabel("Age(years)")#, fontsize=14)

plt.subplot(1,2,2)
plt.title("Chronic Pain Patients")#, fontsize=16)
df_subj_chronic_pain[df_subj_chronic_pain["Sex (m/f)"]=="m"]["Age(years)"].hist(
    histtype="step", label="male", color="b", linewidth=3, density=False)
df_subj_chronic_pain[df_subj_chronic_pain["Sex (m/f)"]=="f"]["Age(years)"].hist(
    histtype="step", label="female", color="r", linewidth=3, density=False)
plt.xlabel("Age(years)")#, fontsize=14)
plt.legend()
plt.show()


#### Plot age distributions for every clinical condition and sex

In men, some ages are not represented

In [ ]:
conditions = df_subj_chronic_pain["Project prefix"].unique()

plt.figure(figsize=(20,8))

for i, condition in enumerate(conditions):
    plt.subplot(1,3,i+1)
    
    df_subj_by_cond = df_subj_chronic_pain[df_subj_chronic_pain["Project prefix"]==condition]
    is_male = df_subj_by_cond["Sex (m/f)"]=="m"
    is_female = is_male.map(lambda x: not x)
    
    plt.title(f"{condition} (n = {is_male.sum()}m/{is_female.sum()}f)")
    df_subj_by_cond[is_male]["Age(years)"].hist(
        histtype="step", label="male", color="b", linewidth=3, density=True)
    df_subj_by_cond[is_female]["Age(years)"].hist(
        histtype="step", label="female", color="r", linewidth=3, density=True)
    plt.xlabel("Age(years)")
    
plt.legend()
plt.show()

#### Plot age distributions for every clinical condition

The age distribution is similar across clinical conditions

In [ ]:
conditions = df_subj_chronic_pain["Project prefix"].unique()


plt.figure(figsize=(8,3))
plt.title("Age distribution for clinical classes")

df_subj_healthy["Age (years)"].hist(
    histtype="step", label=f"Healthy (n = {len(df_subj_healthy)})", linewidth=3, density=False)


for i, condition in enumerate(conditions):
    
    df_subj_by_cond = df_subj_chronic_pain[df_subj_chronic_pain["Project prefix"]==condition]
    
    df_subj_by_cond["Age(years)"].hist(
        histtype="step", label=f"{condition} (n = {len(df_subj_by_cond)})", linewidth=3, density=False)
    plt.xlabel("Age(years)")

plt.ylabel("# participants")
plt.legend(loc="upper left")

In [ ]:
pd.plotting.scatter_matrix(df_subj, figsize=(15,15))

### Analyze a single eeg file

#### Load the file

In [ ]:
example_raw = mne.io.read_raw_brainvision(eeg_files_chronic_pain[0])
channel_to_channel_type = {ch_name:"eeg" for ch_name in example_raw.ch_names}
channel_to_channel_type.update({"LE":"misc", "RE":"misc"})
example_raw.set_channel_types(channel_to_channel_type)

#### Check the metadata

In [ ]:
example_raw

#### Plot the locations of the eeg sensors on the head

Find a montage that fits our data

In [ ]:
for montage_name in mne.channels.get_builtin_montages():
    montage = mne.channels.make_standard_montage(montage_name)
    n_common_chs = len(set(example_raw.ch_names).intersection(set(montage.ch_names)))
    print(
        f"montage: {montage_name} \
        \nn_common_chs out of {len(montage.ch_names)}: {n_common_chs} \
        \n==================")

Set the montage

In [ ]:
montage = mne.channels.make_standard_montage("standard_1020")
set(example_raw.ch_names) - set(montage.ch_names), \
set(montage.ch_names) - set(example_raw.ch_names)
example_raw = example_raw.set_montage(montage)
example_raw.plot_sensors(show_names=True)

#### Visualize a the time and frequency domain

In [ ]:
example_raw.plot()

#### Visualize the frequency domain

In [ ]:
example_raw.compute_psd().plot()

In [ ]:
example_raw.compute_psd().plot_topomap()

### Analyze all files with respect to age

In [ ]:
def get_subj_id(eeg_file: pathlib.Path):
    return eeg_file.stem.split("_")[-3]

def get_age(df_subj, eeg_file: pathlib.Path):
    subj_id = get_subj_id(eeg_file)
    return df_subj[df_subj["Subject ID"] == subj_id]["Age(years)"]
    

#### Color PSDs by age

In [ ]:
# chs_pick = np.random.choice(example_raw.ch_names, replace=False, size=1)
chs_pick = [ch for ch in example_raw.ch_names if not (ch=="LE" or ch=="RE")]

c_scale = 1
min_age = df_subj["Age(years)"].min()
max_age = df_subj["Age(years)"].max()

plt.figure(figsize=(8,8))
plt.xlabel("Frequency")
plt.ylabel("Power Spectral Density (dB/Hz)")
np.random.shuffle(eeg_files_preprocessed)
for f in eeg_files_preprocessed[:20]:
    age = get_age(df_subj, f).values[0]
    raw = mne.io.read_raw_brainvision(f, verbose=False, preload=True)
    
    for i, ch_pick in enumerate(chs_pick):
        raw = raw.set_channel_types(channel_to_channel_type, verbose=False)
        raw = raw.notch_filter(freqs=50, notch_widths=0.5)
        raw = raw.filter(l_freq=1, h_freq=100)
        raw_data = raw.get_data(ch_pick, verbose=False)[0]
        
        plt.subplot(8, 8, i+1)
        plt.title(ch_pick, fontsize=8)
        c_idx = c_scale*(age-min_age)/(max_age-min_age)
        plt.psd(raw_data, color=mpl.colormaps["viridis"](c_idx))
        
        plt.xlabel("")
        plt.ylabel("")

        if i % 8 != 0:
            plt.yticks(np.arange(0, 1, 0.2), 5*[""])
        else:
            plt.yticks(fontsize=6)

        plt.grid(False)
        
# plt.colorbar()

plt.tight_layout()
plt.show()
    

In [ ]:
psds_healthy = []
ages_healthy = []
psds_chronic_pain = []
ages_chronic_pain = []
for f in eeg_files_preprocessed:
    age = get_age(df_subj, f).values[0]
    raw = mne.io.read_raw_brainvision(f, verbose=False, preload=True)
    raw = raw.set_channel_types(channel_to_channel_type, verbose=False)
    raw = raw.notch_filter(freqs=50, notch_widths=0.5)
    raw = raw.filter(l_freq=1, h_freq=100)

    subj_data = df_subj[df_subj["Subject ID"] == get_subj_id(f)]
    is_healthy = subj_data["Pain duration(months)"].isnull().item()
    if is_healthy:
        ages_healthy.append(age)
        psds_healthy.append(np.log(raw.compute_psd().get_data()))
    else:
        ages_chronic_pain.append(age)
        psds_chronic_pain.append(np.log(raw.compute_psd().get_data()))
    
psds_healthy = np.stack(psds_healthy)
ages_healthy = np.array(ages_healthy)

psds_chronic_pain = np.stack(psds_chronic_pain)
ages_chronic_pain = np.array(ages_chronic_pain)

#### Plot PSDs by patient status

In [ ]:
# chs_pick = np.random.choice(example_raw.ch_names, replace=False, size=1)
chs_pick = [ch for ch in example_raw.ch_names if not (ch=="LE" or ch=="RE")]

c_scale = 1
min_age = df_subj["Age(years)"].min()
max_age = df_subj["Age(years)"].max()

plt.figure(figsize=(8,10))
plt.xlabel("Frequency")
plt.ylabel("Power Spectral Density (dB/Hz)")
np.random.shuffle(eeg_files_preprocessed)
# for f in eeg_files_preprocessed[:20]:
#     is_patient = ~df_subj[df_subj["Subject ID"]== get_subj_id(f)].isnull()
#     age = get_age(df_subj, f).values[0]
#     raw = mne.io.read_raw_brainvision(f, verbose=False)
    
for i, ch_pick in enumerate(chs_pick):

    psds_healthy_mean = psds_healthy[:, i, :].mean(0)
    psds_healthy_std = psds_healthy[:, i, :].std(0)
    psds_chronic_pain_mean = psds_chronic_pain[:, i, :].mean(0)
    psds_chronic_pain_std = psds_chronic_pain[:, i, :].std(0)
    
    plt.subplot(8, 8, i+1)
    plt.title(ch_pick, fontsize=8)
    plt.plot(np.linspace(0, 300, 129), psds_healthy_mean, label="Healthy", alpha=0.75)
    plt.fill_between(
        np.linspace(0, 300, 129), 
        psds_healthy_mean - psds_healthy_std, 
        psds_healthy_mean + psds_healthy_std, 
        alpha=0.2
    )
    plt.plot(np.linspace(0, 300, 129), psds_chronic_pain[:, i, :].mean(0), label="Patients", alpha=0.75)
    plt.fill_between(
        np.linspace(0, 300, 129), 
        psds_chronic_pain_mean - psds_chronic_pain_std, 
        psds_chronic_pain_mean + psds_chronic_pain_std, 
        alpha=0.2
    )
    plt.xlabel("")
    plt.ylabel("")

    if i % 8 != 0:
        plt.yticks(np.arange(0, 1, 0.2), 5*[""])
    else:
        plt.yticks(fontsize=6)
    
    plt.xticks(np.arange(0, 300, 100), fontsize=6)
    plt.ylim(-42,-25)
    plt.grid(False)
        
plt.tight_layout()
plt.legend(loc=(1.04, 0), fontsize=8)
plt.show()
    

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
ch_name_to_ch_pos = montage.get_positions()["ch_pos"]

pos = [ch_name_to_ch_pos[ch_name] for ch_name in example_raw.ch_names if channel_to_channel_type[ch_name]=="eeg"]
pos = np.stack(pos)

In [ ]:
# band_size = 20
# band_sections = np.arange(band_size, 120, band_size)
band_sections = [4, 8, 12, 30, 100]
band_names = ["Delta", "Theta", "Alpha", "Beta", "Gamma"]
# freq_bands = np.split(psds, band_sections, axis=-1)[:-1]

fig, ax = plt.subplots(3, len(band_sections), figsize=(12,6))
plt.suptitle("Feature Importances\n", fontsize=16)

l_freq = 0
for i in range(len(band_sections)):

    h_freq = band_sections[i]
    is_band = (raw.compute_psd().freqs > l_freq) * \
              (raw.compute_psd().freqs < h_freq)

    X = np.concatenate([
        psds_healthy[:, :, is_band].sum(-1),
        psds_chronic_pain[:, :, is_band].sum(-1)
    ], axis=0)
    y = np.concatenate([
        ages_healthy,
        ages_chronic_pain
    ], axis=0)
   
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    
    rfg = RandomForestRegressor()
    rfg.fit(X_train, y_train)
    r_squared = rfg.score(scaler.transform(X_test), y_test)

    ax[0,0].set_ylabel("All participants\n") if i == 0 else _
    ax[0,i].set_xlabel(f"R²={r_squared:.2f}, freq {l_freq}-{h_freq}Hz")
    ax[0,i].set_title(f"{band_names[i]}")

    mne.viz.plot_topomap(rfg.feature_importances_, pos[:, :2], axes=ax[0,i], show=False)
    l_freq = h_freq

l_freq = 0
for i in range(len(band_sections)):

    h_freq = band_sections[i]
    is_band = (raw.compute_psd().freqs > l_freq) * \
              (raw.compute_psd().freqs < h_freq)

    X, y = psds_healthy[:, :, is_band].sum(-1), ages_healthy
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    
    rfg = RandomForestRegressor()
    rfg.fit(X_train, y_train)
    r_squared = rfg.score(scaler.transform(X_test), y_test)

    ax[1,0].set_ylabel("Healthy controls\n") if i == 0 else _
    ax[1,i].set_xlabel(f"R²={r_squared:.2f}, freq {l_freq}-{h_freq}Hz")
    mne.viz.plot_topomap(rfg.feature_importances_, pos[:, :2], axes=ax[1,i], show=False)
    l_freq = h_freq

l_freq = 0
for i in range(len(band_sections)):


    h_freq = band_sections[i]
    is_band = (raw.compute_psd().freqs > l_freq) * \
              (raw.compute_psd().freqs < h_freq)

    X, y = psds_chronic_pain[:, :, is_band].sum(-1), ages_chronic_pain
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    
    rfg = RandomForestRegressor()
    rfg.fit(X_train, y_train)
    r_squared = rfg.score(scaler.transform(X_test), y_test)

    ax[2,0].set_ylabel("Chronic Pain Patients\n") if i == 0 else _
    ax[2,i].set_xlabel(f"R²={r_squared:.2f}, freq {l_freq}-{h_freq}Hz")
    mne.viz.plot_topomap(rfg.feature_importances_, pos[:, :2], axes=ax[2,i], show=False)
    l_freq = h_freq

    
plt.tight_layout()
plt.show()

In [ ]:
band_size = 20
band_sections = np.arange(band_size, 120, band_size)
# freq_bands = np.split(psds, band_sections, axis=-1)[:-1]

fig, ax = plt.subplots(len(band_sections), 2 , figsize=(6,15))
plt.suptitle("Feature Importances")

l_freq = 0
for i in range(len(band_sections)):


    h_freq = band_sections[i]
    is_band = (raw.compute_psd().freqs > l_freq) * \
              (raw.compute_psd().freqs < h_freq)

    X, y = psds_healthy[:, :, is_band].sum(-1), ages_healthy
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    
    rfg = RandomForestRegressor()
    rfg.fit(X_train, y_train)
    r_squared = rfg.score(scaler.transform(X_test), y_test)

    ax[0,0].set_title("Healthy controls") if i == 0 else _
    ax[i,0].set_xlabel(f"R² = {r_squared:.2f}, freq {l_freq}-{h_freq} Hz")
    mne.viz.plot_topomap(rfg.feature_importances_, pos[:, :2], axes=ax[i,0], show=False)
    l_freq = h_freq

l_freq = 0
for i in range(len(band_sections)):


    h_freq = band_sections[i]
    is_band = (raw.compute_psd().freqs > l_freq) * \
              (raw.compute_psd().freqs < h_freq)

    X, y = psds_chronic_pain[:, :, is_band].sum(-1), ages_chronic_pain
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    
    rfg = RandomForestRegressor()
    rfg.fit(X_train, y_train)
    r_squared = rfg.score(scaler.transform(X_test), y_test)

    ax[0,1].set_title("Chronic Pain Patients") if i == 0 else _
    ax[i,1].set_xlabel(f" R² = {r_squared:.2f}, freq {l_freq}-{h_freq} Hz")
    mne.viz.plot_topomap(rfg.feature_importances_, pos[:, :2], axes=ax[i,1], show=False)
    l_freq = h_freq

    
plt.tight_layout()
plt.show()